In [1]:
# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 生成實體物件
# line_bot_api = LineBotApi()
# handler = WebhookHandler()

In [2]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    SourceUser, SourceGroup, SourceRoom,
    TemplateSendMessage, ConfirmTemplate, MessageAction,
    ButtonsTemplate, ImageCarouselTemplate, ImageCarouselColumn, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    CarouselTemplate, CarouselColumn, PostbackEvent,
    StickerMessage, StickerSendMessage, LocationMessage, LocationSendMessage,
    ImageMessage, VideoMessage, AudioMessage, FileMessage,
    UnfollowEvent, FollowEvent, JoinEvent, LeaveEvent, BeaconEvent,
    FlexSendMessage, BubbleContainer, ImageComponent, BoxComponent,
    TextComponent, SpacerComponent, IconComponent, ButtonComponent,
    SeparatorComponent, QuickReply, QuickReplyButton
)
from linebot.models import (
    MessageEvent, PostbackEvent, 
    TextSendMessage, TemplateSendMessage,
    TextMessage, ButtonsTemplate,
    PostbackTemplateAction, MessageTemplateAction,
    URITemplateAction, 
)

In [3]:
# 載入json處理套件
import json

# 載入基礎設定檔
secretFile=json.load(open("./secret_key",'r'))
server_url=secretFile.get("server_url")


# channel_access_token是用於傳送封包去給line的認證使用類似這個是私鑰，而公鑰已經丟到line那邊，拿著這個就可以有VIP特權
# secret_key是用於當line傳送封包過來時確定line是否為本尊，沒有被仿冒
line_bot_api = LineBotApi(secretFile.get("channel_access_token"))
handler = WebhookHandler(secretFile.get("secret_key"))
menu_id = secretFile.get("rich_menu_id")


In [4]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="CC102-Line考古題機器人。")
]
    

In [5]:
"""

  啟用伺服器基本樣板啟用伺服器基本 

"""

# 引用Web Server套件
from flask import Flask, request, abort , jsonify



# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [6]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    ################### 須作例外處理
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊做成合適Json

    user_info = {  
        "user_open_id":user_profile.user_id,
        "user_nick_name":user_profile.display_name,
        "user_status" : "",
        "user_img" : user_profile.picture_url,
        "user_register_menu" : menu_id
    }
    #將json傳回API Server
    Endpoint='http://192.168.122.100/users'   
    Header={'Content-Type':'application/json'}
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(user_info))
    
    # 先暫時寫的，用來檢查錯誤情形，方便除錯
    print(Response)
    print(Response.text)
    
    # 將菜單綁定在用戶身上
    
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (user_profile.user_id, menu_id)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFile["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    print(lineLinkMenuResponse)
    print(lineLinkMenuResponse.text)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
         event.reply_token,
         reply_message_list
    )

In [7]:
  
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
@handler.add(PostbackEvent)
def handle_post_message(event):
    #這邊要再想想怎樣比較好
    a = test_sa(1)
    #假如回答等於正解
    if (event.postback.data == a['true_answer']):
        reply_message=[
            TextMessage(text=a["true_answer_decribe_content"]),
            TextMessage(text="link"+a["external_link"])
        ]
        line_bot_api.reply_message(event.reply_token,reply_message)
    else:
        pass
   


In [13]:

'''
將素材消息做成一本字典，當用戶發出相應消息時，可從此進行查找動作。
'''
#給github功能的回覆訊息
reply_github = [
    TextSendMessage(text="https://github.com/iii-cutting-edge-tech-lab/Chatbot_Project")
    ]
#暫時沒有sys的題目先回一個隨便的話
sys_message_list = [
    TextSendMessage(text="關注區塊鏈技術，掌握市場脈動。")
    ]
#裝關鍵字的字典，會指向專屬訊息或函式
template_message_dict = {
    "[::text:]testsa": testsa(),
    "[::text:]testdev":testdev(),
    "[::text:]testsys":sys_message_list,
    "[::text:]github": reply_github
}


In [9]:
#這個函式方便日後寫有關亂數的函式
def test_sa(qid):
    #api server接口位置
    url =  'http://192.168.122.100/question/sysops'
    #裝query string的部份
    payload = {'question_id' : qid}
    #傳送封包
    a = requests.get(url,params=payload)
    #將回傳結果取出
    a=a.json()
    return a

In [10]:
#做一個回傳dev考題的函式
def testdev():
    #只有sa的題目可用
    a = test_sa(1)
    #這邊使用quick reply的方式
    quickreply = TextSendMessage(
                text='Choose your answer',
                quick_reply=QuickReply(
                    items=[
                        QuickReplyButton(
                            action=PostbackAction(label="A",data="A")
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="B",data="B")
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="C",data="C")
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="D",data="D")
                        )
                    ]))
    #全部回傳的list
    dev_reply_message_list = [
        TextSendMessage(text=a["question_content"]),
        TextSendMessage(text=a["answer1_content"]+"\n\n"+a["answer2_content"]+"\n\n"+a["answer3_content"]+"\n\n"+a["answer4_content"]),
        #回傳quick reply選單
        quickreply    
    ]
    return dev_reply_message_list

In [11]:
#從datebase拿取考題並回覆
def testsa():
    #利用question_id取得考題，之後在寫成亂數id
    a = test_sa(1)
    #做一個ButtonsTemplate選單
    button_template_message =ButtonsTemplate(
                                text='choose your answer',
                                actions=[
                                    PostbackTemplateAction(
                                        label='A',
                                        text = None,
                                        data='A'
                                    ),
                                    PostbackTemplateAction(
                                        label='B', 
                                        text = None,
                                        data='B'
                                    ),
                                    PostbackTemplateAction(
                                        label='C', 
                                        text = None,
                                        data='C'
                                    ),
                                    PostbackTemplateAction(
                                        label='D', 
                                        text = None,
                                        data='D'
                                    ),
                                ]
                            )
    #全部要回傳的訊息
    sys_reply_message_list = [
        #題目
        TextSendMessage(text=a["question_content"]),
        #選項
        TextSendMessage(text=a["answer1_content"]+"\n\n"+a["answer2_content"]+"\n\n"+a["answer3_content"]+"\n\n"+a["answer4_content"]),
        #選單
        TemplateSendMessage(
                alt_text="Template Example",
                template=button_template_message
            )    
    ]
    return sys_reply_message_list

In [12]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
#將這次event的參數抓進來
def handle_message(event):
    #抓關鍵字
    if(event.message.text.find('::text:')!= -1):
        #回覆訊息
        line_bot_api.reply_message(
            #一定要有reply_token，而且只能用一次
            event.reply_token,
            #抓字典內找符合的動作
            template_message_dict.get(event.message.text)
            )
        
    else:
        #針對一些無意義的訊息
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊功能選單上的圖示"))

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.27.0.4 - - [16/Sep/2018 13:31:42] "POST / HTTP/1.1" 200 -
172.27.0.4 - - [16/Sep/2018 13:32:04] "POST / HTTP/1.1" 200 -
172.27.0.4 - - [16/Sep/2018 13:32:30] "POST / HTTP/1.1" 200 -
172.27.0.4 - - [16/Sep/2018 13:32:45] "POST / HTTP/1.1" 200 -
